# Overview

- The strategy introduces **institutional positioning signals** to capture major-holder accumulation trends and enhance Bitcoin accumulation performance.
- The core principle of this strategy is to increase Bitcoin allocation whenever **MicroStrategy** announces a new Bitcoin purchase. The underlying assumption is that MicroStrategy’s accumulation events often lead to upward price movements in BTC. By following these institutional purchase signals—particularly during local price dips—the strategy aims to accumulate more Bitcoin at favorable prices and enhance long-term performance.
- In theory, this kind of positioning strategy should work.

- Institutional positioning signals: indicators that capture the buying or selling behavior of large institutions, whose actions often influence overall market trends.
- MicroStrategy (MSTR): a U.S.-based business intelligence and software company widely known for its aggressive Bitcoin accumulation strategy

# MSTR Signal

- The trading signals come from MSTR’s **8-K reports**, where the company officially announces its Bitcoin purchases to the SEC.
- MicroStrategy began disclosing Bitcoin purchase activity through Form 8-K in 2021. 
- All Bitcoin-related 8-K filings from 2021 to the present have been reviewed, resulting in a total of **72** records.
Each record contains three key fields: **purchase_date, btc_amount, and usd_amount**.
- To align with the availability of MicroStrategy’s 8-K signals, the strategy is evaluated over the period from **2021 to 2025**.

In [2]:
import pandas as pd
pd.read_csv("mstr_buy_signal.csv")

,purchase_date,btc_amount,usd_amount
0,2025/3/17,130,1.070000e+07
1,2025/8/11,155,1.800000e+07
2,2025/10/20,168,1.880000e+07
3,2025/9/29,196,2.210000e+07
4,2021/3/5,205,1.000000e+07
...,...,...,...
67,2024/12/9,21550,2.100000e+09
68,2025/3/31,22048,1.920000e+09
69,2024/11/12,27200,2.030000e+09
70,2024/11/18,51780,4.600000e+09


# Retrain Dynamic DCA Model

- Because the MSTR signal only starts in 2021, this backtest is limited to the 2021–2025 period. The original Dynamic DCA model was trained on 2011–2025 data, and parameters trained on that full range cannot produce optimized results for this shorter backtest window. Therefore, I retrained the Dynamic DCA model using only the 2021–2025 data.
- The newly trained model achieved an Exponential-Decay Average SPD Percentile of 81.75119% and a 100.00% win rate over the 2011–2025 period. Final Model Score (50/50 weighting): 90.88%

# Strategy

- Based on the baseline weights generated by the Dynamic DCA model, the strategy increases the allocation weight during periods when an MSTR signal is detected.
- The goal is to see whether adding MSTR signals can make Dynamic DCA perform better.
    - Exponential-Decay Average SPD Percentile of 81.75% and a 100.00% win rate.
- I test three different strategies.

## Strategy 1

**Intro:**
- For each day, the model looks back over the past *n* days. If *m* MSTR purchase events occurred within that window, the baseline Dynamic DCA weight for that day is multiplied by **(1 + m × k)**.  
- The lookback window *n* is tested with values **7, 14, and 30 days**, and the boost coefficient *k* (signal strength) is tested with **0.2, 0.3, and 0.5**.

**Performance:** 80.71% / 99.79%
- The result is slightly worse than the baseline Dynamic DCA.  
- The weights of both strategies are almost identical in every window, so the MSTR adjustment has little to no effect on performance

**Conclusion:**\
\
It is likely because the `allocate_sequential` method tends to dilute the boosted weights derived from MSTR signals. Since this allocation function normalizes and redistributes weights sequentially from earlier to later days, any local increase caused by MSTR signals is quickly averaged out when the remaining budget is reallocated. As a result, the boosted effect on specific days becomes minimal after normalization.


## Strategy 2

**Intro:**
- To address this issue, the budget for Dynamic DCA and MSTR signals is separated. The MSTR portion is excluded from the `allocate_sequential` process so that the additional weights generated by MSTR events will not be diluted during sequential normalization. This allows the boosted allocation from MSTR signals to take full effect instead of being averaged out by the baseline allocation mechanism.
- The total budget is divided into two sleeves: one for Dynamic DCA and one for MSTR. The Dynamic DCA part follows the same procedure as before, but its weights are multiplied by **(1.0 − SLEEVE_MSTR)**. The weights calculated from MSTR signals are multiplied by **SLEEVE_MSTR**, and the two results are then summed to form the final allocation weights.
- For the MSTR sleeve, the method scans back *n* days for each day. If there are *m* MSTR events within that window, the entire MSTR budget is allocated equally to those *m* event days. All other days only maintain the minimum weight.


**Performance:** 65.75% / 100%
- The performance is much worse than both the baseline Dynamic DCA and Strategy

**Conclusion:** \
\
It is likely that separating the budgets is not a feasible approach.  

## Strategy 3

**Intro:**
- Grouped the MSTR BTC purchase amounts into three levels using Jenks Natural Breaks. Larger purchase amounts correspond to a larger boost factor. For each event day, the Dynamic DCA weight is multiplied by the assigned boost value based on the cluster level.
- Added an exponential decay component to account for the time-lagged influence of each MSTR event. For every event occurring at day *tᵢ*, an exponential decay curve is formed. This means that days immediately after an event also receive a boosted weight. The effect gradually decreases as the distance from the event day increases, reflecting the assumption that an MSTR purchase may influence several subsequent days but with diminishing strength over time. The decay parameter τ is tested with values **20~120 days** to evaluate different decay speeds.
$$
f(d) = 1 + \sum_{t_i \le d} (boost_i - 1) e^{-(d - t_i)/\tau}
$$
- Used the boost cap to limit the maximum amplification applied to Dynamic DCA weights, regardless of the cluster-based boost value or the exponential decay contribution.
- Added a compatibility suppression mechanism to avoid conflicting signals. For example, situations where the price z-score is very high (expensive) but the MSTR event would otherwise boost the allocation. This suppression ensures that price-based overvaluation signals can override or moderate the MSTR-driven boost.\
Defined a smooth-maximum (softmax-style) overheating indicator across three windows (30, 90, 180 days). The final effective boost is scaled by this suppression factor to ensure consistency between price signals and MSTR-based boosts. If $ z_{\text{sup}} $ exceeds the threshold $z_{\text{clip}} = 3$, the boost is effectively disabled. In other words, once $ z_{\text{sup}} > 3 $, no additional MSTR boost should be applied.

$$
z_{\text{sup}} = 
\max\left(0,\;
T \cdot \log\left(
\frac{e^{z_{30}/T} + e^{z_{90}/T} + e^{z_{180}/T}}{3}
\right)\right)
$$


$$
\text{supp} = \max\left(0,\; 1 - \frac{z_{\text{clip}}}{z_{\text{sup}}} \right)
$$


$$
z_{\text{clip}} = 3.0000
$$

**Performance:** 82.9% / 100%
- Average SPD Percentile is higher than Dynamic DCA's by 1.2％

**Conclusion**
- Although the performance is slightly better than the original Dynamic DCA, the improvement is very limited, making it hard to conclude whether incorporating the MSTR signal is actually effective.